In [ ]:
# 기본
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.externals import joblib 
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install catboost
!pip install category_encoders

In [ ]:
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

In [ ]:
# 경고창 무시 -> 회사에서 업무 처리할때는 무시하면 안됨
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive'

**1. 주제**
- **신용카드 사용자 데이터를 보고 사용자의 대금 연체 정도를 예측**하는 알고리즘 개발

**2. 배경**
- 신용카드사는 신용카드 신청자가 제출한 개인정보와 데이터를 활용해 신용 점수를 산정
- 신용카드사는 이 신용 점수를 활용해 신청자의 향후 채무 불이행과 신용카드 대급 연체 가능성을 예측

**3. 목표**
- 사용자의 대금 연체 정도를 예측할 수 있는 인공지능 알고리즘을 개발해 금융업계에 제안할 수 있는 인사이트를 발굴

In [ ]:
# 데이터 불러오기
df_train = pd.read_csv('/content/drive/My Drive/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

# **1. EDA**

## **데이터 타입 및 변수 확인**

**Binary**

- gender: 성별

- car: 차량 소유 여부

- reality: 부동산 소유 여부

- FLAG_MOBIL: 핸드폰 소유 여부(가정용)

- work_phone: 업무용 전화 소유 여부

- phone: 전화 소유 여부

- email: 이메일 소유 여부

**Multi_category**

- house_type: 생활 방식

- occyp_type: 직업 유형

- income_type: 소득 분류

- family_type: 결혼 여부

**discrete type**

- family_size: 가족 일원 수 

- child_num : 자녀 수

**ordinary type**

- edu_type : 교육 수준

- credit : 신용 기준 → 낮을 수록 높은 시용의 신용카드 사용자(대금연체를 기준으로 한 신용도임)

**continuus type**

- income_total : 수익

- DAYS_BIRTH : 출생일 → 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미

- DAYS_EMPLOYED : 업무 시작일 →  -1은 데이터 수집일 하루 전부터 일을 시작함을 의미 / 양수 값은 고용되지 않은 상태를 의미함

- begin_month : 신용카드 발급 월 → -1은 데이터 수집일 한 달 전에 신용카드를 발급함 / 데이터 수집 당시 0부터 역으로 셈




## **결측치 확인**

In [ ]:
df_train.isnull().sum()

train 데이터의 컬럼 정보를 살펴보았을때 occyp_type 변수만 결측치가 존재하는 것을 확인

직업 유형이 결측치 -> 직업이 없는 무직자로 생각해야할지 or 전체 데이터 중 30% 비율의 결측치를 제거해야할지 고민이 필요하다.


In [ ]:
df_train['credit'].value_counts()

## **Credit (종속변수 - 예측값) 비율 살펴보기**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,6))
df_train['credit'].value_counts().plot.pie(explode=[0,0.1,0.1],autopct='%1.1f%%',ax=axes[0], textprops={'fontsize': 14},shadow=True,colors=['lightsteelblue','cornflowerblue','royalblue'])
axes[0].set_title('credit')
axes[0].set_ylabel('credit')
sns.countplot('credit',data=df_train,ax=axes[1],palette='Set2')
axes[1].set_title('credit')
plt.show()

2→1→0으로 갈수록 신용등급이 높음

그러나 그래프를 살펴보았을때, 신용등급이 낮은 고객들의 수가 더 높은 것을 확인 할 수 있다.

## **Binary**

- 전반적으로 크게 드러나는 특징은 존재하지 않음

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# 성별
sns.distplot(df_train[df_train['gender']=='M']['credit'], ax=axes[0],color='blue')
sns.distplot(df_train[df_train['gender']=='F']['credit'], ax=axes[0],color='red')

# 차
sns.distplot(df_train[df_train['car']=='N']['credit'], ax=axes[1],color='lightsteelblue')
sns.distplot(df_train[df_train['car']=='Y']['credit'], ax=axes[1],color='dodgerblue')

# 부동산
sns.distplot(df_train[df_train['reality']=='N']['credit'], ax=axes[2],color='lightsteelblue')
sns.distplot(df_train[df_train['reality']=='Y']['credit'], ax=axes[2],color='dodgerblue')

axes[0].set_title('gender')
axes[1].set_title('car')
axes[2].set_title('reality')

In [ ]:
pd.crosstab(df_train.credit,df_train.gender,margins=True).style.background_gradient(cmap='YlGnBu') # 성별

In [ ]:
pd.crosstab(df_train.credit,df_train.car,margins=True).style.background_gradient(cmap='YlGnBu') # 차

In [ ]:
pd.crosstab(df_train.credit,df_train.reality,margins=True).style.background_gradient(cmap='YlGnBu') # 부동산

1) 성별 
- 남성, 여성 모두 신용점수가 낮은 사람들이 높은 사람들보다 많은 것을 확인할 수 있다.
- 남성보다는 여성이 더 많음

2) 차량 소유 여부
- 차의 소유 여부와 상관없이 신용등급이 낮은 사람들이 더 높은 것을 확인
- 차가 없는 사람들이 더 많음

3) 부동산 소유 여부
- 부동산 소유 여부와 상관없이 신용등급이 낮은 사람들이 더 높은 것을 확인
- 부동산을 소유한 사람들이 더 많음


In [ ]:
# 핸드폰 소유 여부(가정용)
set(list(df_train['FLAG_MOBIL'])) # 1만 나옴 가정용 핸드폰은 모두 존재..? 그러므로 이 변수는 제거해도 될것 같음

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# 업무용 전화소유 여부
sns.distplot(df_train[df_train['work_phone']==0]['credit'], ax=axes[0],color='lightsteelblue')
sns.distplot(df_train[df_train['work_phone']==1]['credit'], ax=axes[0],color='dodgerblue')

# 전화소유 여부
sns.distplot(df_train[df_train['phone']==0]['credit'], ax=axes[1],color='lightsteelblue')
sns.distplot(df_train[df_train['phone']==1]['credit'], ax=axes[1],color='dodgerblue')

# 이메일 소유 여부
sns.distplot(df_train[df_train['email']==0]['credit'], ax=axes[2],color='lightsteelblue')
sns.distplot(df_train[df_train['email']==1]['credit'], ax=axes[2],color='dodgerblue')

axes[0].set_title('work_phone')
axes[1].set_title('phone')
axes[2].set_title('email')

In [ ]:
pd.crosstab(df_train.credit,df_train.work_phone,margins=True).style.background_gradient(cmap='YlGnBu') # 업무용 전화 

In [ ]:
pd.crosstab(df_train.credit,df_train.phone,margins=True).style.background_gradient(cmap='YlGnBu') # 전화 

In [ ]:
pd.crosstab(df_train.credit,df_train.email,margins=True).style.background_gradient(cmap='YlGnBu') # 이메일 

4) 가정용 핸드폰 
- 데이터 값 1만 존재 → 가정용 핸드폰은 모두 존재..? 
그러므로 이 변수는 신용등급 변화에 유의미한 변화를 줄 것같지는 않음


5) 업무용 전화 
- 업무용 전화가 존재하든 안하든 모두 신용점수가 낮은 사람들이 높은 사람들보다 많은 것을 확인할 수 있다.
- 업무용 전화가 없는 사람이 더 많음

2) 전화 소유 여부
- 전화의 소유 여부와 상관없이 신용등급이 낮은 사람들이 더 높은 것을 확인
- 전화가 없는 사람들이 더 많음

3) 이메일 소유 여부
- 이메일의 소유 여부와 상관없이 신용등급이 낮은 사람들이 더 높은 것을 확인
- 이메일을 소유하지 않은 사람들이 더 많음

## **Muti Category**

특이한 점은 발견할 수 없었음

**house_type - 생활방식**

In [ ]:
pd.crosstab(df_train.credit,df_train.house_type,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot('house_type',hue='credit',data=df_train,palette='husl')

테이블과 그래프를 살펴보았을때, 평범한 아파트에서 사는 사람들이 대부분인 것을 확인 할 수 있다.

**occyp_type - 직업유형**

In [ ]:
pd.crosstab(df_train.credit,df_train.occyp_type,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot('occyp_type',hue='credit',data=df_train,palette='husl')

HR staff, Low-skill Laborers, Realty agents는 신용등급이 1인사람들보다 등급이 0인 사람들이 더 많은 것을 확인 할 수 있다.

**income_type - 소득분류**

In [ ]:
pd.crosstab(df_train.credit,df_train.income_type,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot('income_type',hue='credit',data=df_train,palette='husl')

데이터에서 working이 차지하는 비율이 높다는 것을 확인 할 수 있음

소득분류에서 학생이 차지하는 부분은 매우 적은것을 확인 할 수 있다.

**family_type - 결혼여부**

In [ ]:
pd.crosstab(df_train.credit,df_train.family_type,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot('family_type',hue='credit',data=df_train,palette='husl')

결혼한 사람들의 비율이 가장 높다는 것을 확인 할 수 있다.

## **discrete type**

**family_size  - 가족 일원 수**

In [ ]:
pd.crosstab(df_train.credit,df_train.family_size,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot('family_size',hue='credit',data=df_train,palette='husl')

가족 일원수가 2명인 곳이 신용등급의 차가 매우 큰 것을 알 수 있다. 
ex) 1→2로 등급이 나빠지는 폭이 매우 커짐

대가족인 데이터들이 간혹 보이지만, 신용등급이 좋지는 않음

**child_num - 자녀 수**

In [ ]:
pd.crosstab(df_train.credit,df_train.child_num,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot('child_num',hue='credit',data=df_train,palette='husl')

자녀 수가 0인 사람들이 대부분인것을 확인 할 수 있다.
신용등급이 안좋은 사람들이 극단적으로 많음

가족 일원 수와 비슷한 분포를 지닌것을 확인 할 수 있다.

## **ordinary type**

**edu_type - 교육수준**

In [ ]:
pd.crosstab(df_train.credit,df_train.edu_type,margins=True).style.background_gradient(cmap='YlGnBu') 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,6))
df_train['edu_type'].value_counts().plot.pie(explode=[0,0.1,0.1,0.1,0.1],autopct='%1.1f%%',ax=axes[0], textprops={'fontsize': 14},shadow=True,colors=['lightcoral','orange','turquoise','cornflowerblue','mediumpurple'])
axes[0].set_title('edu_type')
axes[0].set_ylabel('edu_type')
sns.countplot('edu_type',hue='credit',data=df_train,palette='husl')
axes[1].set_title('edu_type')
plt.show()

전체 데이터에서 Secondary가 가장 높은 비율을 차지하는 것을 확인 할 수 있다. 

## **continuous type**

**income_total - 수익**


In [ ]:
plt.figure(figsize=(15,10))

sns.boxplot(x = "credit", y = "income_total", data = df_train,palette = "husl")

plt.show()

신용등급이 높을수록 수익의 격차가 적은 것을 확인 할 수 있다.

박스플랏을 살펴보면 신용등급이 높을수록? 좋을 수록? 매우 미세한 정도로 수익이 높은 것을 확인 할 수 있다.

신용등급이 낮은 사람들의 수익을 살펴보았을 때 빈부격차가 매우 심한것을 확인 할 수 있다. 

수익이 신용등급에 어느정도 조금?은 영향을 미치는 것으로 확인된다.

**DAYS_BIRTH - 출생일**

In [ ]:
# 일 단위를 연 단위로 변환
df_train.loc[:,'age']=abs(df_train['DAYS_BIRTH']/365).astype(int)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

sns.distplot(df_train[df_train['credit']==0]['age'], ax=axes[0],color='cornflowerblue')
sns.distplot(df_train[df_train['credit']==1]['age'], ax=axes[1],color='cornflowerblue')
sns.distplot(df_train[df_train['credit']==2]['age'], ax=axes[2],color='cornflowerblue')

sns.regplot(y=df_train['age'], x=df_train['income_total'],ax=axes[3],fit_reg=True,color='lightcoral')
axes[0].set_title('credit_0')
axes[1].set_title('credit_1')
axes[2].set_title('credit_2')

신용등급이 좋은 사람들의 나이를 살펴보았을 때, 30-40대 초반에 높은 것을 확인 할 수 있다. 30-40에 보통 정기적으로 수익을 가지고 직업을 가지기 때문에 이러한 결과가 나온 것으로 예상

신용등급이 보통인 사람들도 등급이 높은 사람들과 비슷한 결과가 나온것을 확인 할 수 있다.

반면, 신용등급이 낮은 사람들은 나이대가 무분별한것을 확인 할 수 있다.


수익을 살펴보았을때 특정나이대에 수익이 높은것은 아닌것 같다.

**DAYS_EMPLOYED - 업무시작일**

In [ ]:
 df_train['DAYS_EMPLOYED'].hist()

무직자가 생각보다 많은것을 확인 할 수 있다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

sns.distplot(df_train[df_train['DAYS_EMPLOYED']>=0]['credit'],ax=axes[0],color='dodgerblue')
sns.distplot(df_train[df_train['DAYS_EMPLOYED']<0]['credit'],ax=axes[1],color='lightcoral')
axes[0].set_title('no_working')
axes[1].set_title('working')


무직자랑 직업이 있는 사람들의 분포 차이가 거의 없는 것을 확인 할 수 있다.

**begin_month - 신용카드 발급 월**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

sns.distplot(abs(df_train[df_train['credit']==0]['begin_month']), ax=axes[0],color='cornflowerblue')
sns.distplot(abs(df_train[df_train['credit']==1]['begin_month']), ax=axes[1],color='cornflowerblue')
sns.distplot(abs(df_train[df_train['credit']==2]['begin_month']), ax=axes[2],color='cornflowerblue')

axes[0].set_title('credit_0')
axes[1].set_title('credit_1')
axes[2].set_title('credit_2')

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(abs(df_train[df_train['credit']==0]['begin_month']),label='credit0')
sns.distplot(abs(df_train[df_train['credit']==1]['begin_month']),label='credit1')
sns.distplot(abs(df_train[df_train['credit']==2]['begin_month']),label='credit2')
plt.legend()

신용등급이 낮을수록 신용카드를 더 많이 사용하는 경향이 있음 (발급월이 늘어날 수록 그만큼 신용카드를 오래 사용했다는 것을 추측)

현재 우리나라에서는 신용카드를 많이 사용하면 신용등급이 올라가기때문에 신용카드를 많이 사용하는데, 이는 데이터 출처 나라에도 적용되기 때문에 등급이 낮은 사람들이 조금더 신용카드를 사용하지 않았을까 하는 생각

# **※1등 코드를 참조하여 공부 + pycaret 시도**

1등 코드 출처 : https://dacon.io/competitions/official/235713/codeshare/2768?page=1&dtype=recent

pycaret 코드 출처 : https://dacon.io/competitions/official/235689/codeshare/2375?page=3&dtype=recent&ptype=pub
-> 실패

# **2. 데이터 전처리**

## **결측치 제거**

In [ ]:
df_train.isnull().sum()

In [ ]:
(8171/len(df_train))*100

전체 데이터 중 결측치가 30%나 차지하기때문에 조정 필요

In [ ]:
# 결측치 처리
df_train.fillna('NaN', inplace=True) 
df_test.fillna('NaN', inplace=True)

## **이상치 처리**

- family_size >= 7 인 데이터 제거하기 / test 데이터에는 없기 때문

In [ ]:
df_train = df_train[(df_train['family_size'] < 7)]
df_train = df_train.reset_index(drop=True)

## **Feature  Engineering**

1) 필요없는 변수 제거

- index
- FLAG_MOBIL의 값들이 모두 1이므로 삭제

In [ ]:
df_train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
df_test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

2) DAYS_EMPLOYED 양수 값은 고용되지 않은 상태를 의미함 그러므로 양수는 0처리

In [ ]:
df_train['DAYS_EMPLOYED']=df_train['DAYS_EMPLOYED'].map(lambda x: 0 if x>0 else x)
df_test['DAYS_EMPLOYED']=df_test['DAYS_EMPLOYED'].map(lambda x: 0 if x>0 else x)

3) DAYS_BIRTH, begin_month, DAYS_EMPLOYED 헷갈리지 않게 양수로 변환

In [ ]:
variables = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for variable in variables:
    df_train[variable]=np.abs(df_train[variable])
    df_test[variable]=np.abs(df_test[variable])

4) 파생변수 생성

In [ ]:
for df in [df_train,df_test]:
    # before_EMPLOYED: 고용되기 전까지의 일수 -> 언제 취업 했는지 나이대가 언제 인지 추측가능할듯 
    df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
    df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']
    df['before_EMPLOYED_Month'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['before_EMPLOYED_Week'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)
    
    #DAYS_BIRTH 파생변수- Age(나이), 태어난 월, 태어난 주(출생연도의 n주차)
    df['Age'] = df['DAYS_BIRTH'] // 365
    df['DAYS_BIRTH_Month'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
    df['DAYS_BIRTH_Week'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)

    
    #DAYS_EMPLOYED_m 파생변수- EMPLOYED(근속연수), DAYS_EMPLOYED_m(고용된 달) ,DAYS_EMPLOYED_w(고용된 주(고용연도의 n주차))  
    df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
    df['DAYS_EMPLOYED_Month'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['DAYS_EMPLOYED_Week'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

    #ability: 소득/(살아온 일수+ 근무일수) -> ? 이거는 무슨 의미인지 잘 모르겠음
    df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])
    
    #income_mean: 소득/ 가족 수
    df['income_mean_fa'] = df['income_total'] / df['family_size']
    
    #ID 생성: 각 컬럼의 값들을 더해서 고유한 사람을 파악(*한 사람이 여러 개 카드를 만들 가능성을 고려해 begin_month는 제외함) - 중복되는 데이터 때문에 생성한것 같음
    df['ID'] = \
    df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
    df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
    df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
    df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
    df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
    df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
    df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
    df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)

In [ ]:
df_train

In [ ]:
len(df_train)

5) 파생변수와 다중공선성을 보이는 컬럼 삭제 

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data = df_train.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
cols = ['child_num','DAYS_BIRTH', 'DAYS_EMPLOYED']
df_train.drop(cols, axis=1, inplace=True)
df_test.drop(cols, axis=1, inplace=True)

## **Scaling, Encoding**

1) 범주형, 연속형 비교

In [ ]:
numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
categorical_feats

2) 로그 변환

- income_total

In [ ]:
df_train.income_total.hist()

In [ ]:
# 원본에 1을 더한 값에 로그를 씌우면 결국 log1인 것이니 0부터 값이 잘 나오게 된다. 그러나 매번 1을 더하기는 귀찮으니 Numpy에서는 log1p라는 함수로 이를 한번에 해줄 수 있다.
for df in [df_train,df_test]:
    df['income_total'] = np.log1p(1+df['income_total'])

3) Encoding 범주형 -> 숫자

In [ ]:
encoder = OrdinalEncoder(categorical_feats)
df_train[categorical_feats] = encoder.fit_transform(df_train[categorical_feats], df_train['credit'])
df_train['ID'] = df_train['ID'].astype('int64')

In [ ]:
encoder = OrdinalEncoder(categorical_feats)
df_test[categorical_feats]=encoder.fit_transform(df_test[categorical_feats])
df_test['ID'] = df_test['ID'].astype('int64')

4) Clustering
- 타겟을 결정 짓는 뚜렷한 특징을 갖는 피처를 찾기 위해 군집시도

In [ ]:
kmeans_train = df_train.drop(['credit'], axis=1)
kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
df_train['cluster'] = kmeans.predict(kmeans_train)
df_test['cluster'] = kmeans.predict(df_test)

5) StandardScale
- 이미 로그변환을 진행한 income_total을 제외한 나머지 numeric 컬럼 정규화

In [ ]:
numerical_feats.remove('income_total')
scaler = StandardScaler()
df_train[numerical_feats] = scaler.fit_transform(df_train[numerical_feats])
df_test[numerical_feats] = scaler.transform(df_test[numerical_feats])

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## **Modeling - Catboost**

In [ ]:
n_est = 2000
seed = 42
n_fold = 15
n_class = 3

target = 'credit'
X = df_train.drop(target, axis=1)
y = df_train[target]
X_test = df_test

In [ ]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'ID']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')

# **변수 중요도**

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(model_cat.get_feature_importance(),X_test.columns,'CATBOOST')

# **Submission**

In [ ]:
sub = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

In [ ]:
sub.iloc[:, 1:] = cat_pred_test

# **Pycaret**

In [ ]:
! pip install pycaret

In [ ]:
### Find Best Model with Pycaret
from pycaret.classification import *

In [ ]:
df_train

In [ ]:
df_train.credit=df_train.credit.astype('category')

In [ ]:
# 범주형 변수들에서만 pycaret이 적용되는 것 같음

check=df_train[['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type',
 'ID','credit']]

In [ ]:
clf = setup(data = check, target = 'credit', silent = True)

In [ ]:
compare_models(sort = 'Accuracy', n_select = 1, fold = 10) 

# 실패 ㅠㅠㅠㅠ 